# Проверочный проект по линейной регрессии

Мы изучили построение признаков, кросс-валидацию и поиск по сетке. Пришло время проверить Ваши новые навыки. Добро пожаловать на проверочный проект по машинному обучению! В этом проекте у нас будут достаточно подробные пошаговые инструкции. Будущие проекты будут уже в более свободной форме. Мы начнём проект с финальной версии набора данных Ames Housing, с которым мы работали в разделе "построение признаков" этого курса. Цель этого проекта - создать модель линейной регрессии, обучить её на данных с поиском оптимальных значений параметров по сетке, и затем оценить модель на тестовом наборе данных.

---
---
---
## Выполните задания, написанные жирным шрифтом

**ЗАДАНИЕ: Выполните ячейки ниже для импорта библиотек и загрузки данных. Возможно в будущем Вам понадобятся дополнительные команды import от scikit-learn.**

### Imports

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Данные

In [19]:
df = pd.read_csv("AMES_Final_DF.csv")

In [20]:
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


In [22]:
df['SalePrice']

0       215000
1       105000
2       172000
3       244000
4       189900
         ...  
2920    142500
2921    131000
2922    132000
2923    170000
2924    188000
Name: SalePrice, Length: 2925, dtype: int64

**ЗАДАНИЕ: Мы будем пытаться спрогнозировать значение колонки SalePrice. Разделите данные на две части - признаки X и целевая переменная y.**

In [23]:
## Создаём X и y
X = df.drop('SalePrice',axis=1)
y = df['SalePrice']

**ЗАДАНИЕ: С помощью scikit-learn разделите и X, и y на обучающий и тестовый наборы данных. Поскольку далее мы будем использовать поиск по сетке, то выделите под тестовые данные 10% от всех данных. Чтобы получить такое же разбиение данных, как и в нашем блокноте, можете использовать random_state = 101.**

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)


**ЗАДАНИЕ: Признаки в этом наборе данных имеют различный масштаб и различные единицы измерения. Для оптимальной работы регрессии, выполните масштабирование признаков X. Обратите внимание на то, какие данные подавать на вход для .fit(), а какие данные на вход для .transform().**

In [25]:
# Масштабирование данных - SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

По-хорошему такой методики нужно придерживаться всегда. Обучение любых моделей, включая такие прострые штуки как скейлеры, импутеры, нужно проводить исключительно на тренировочных данных. Модель не должна видеть тестовой и валидационной части данных на этапе обучения. Даже если речь о заполнении пропусков или масштабировании. Любое нарушение этого правила рано или поздно закончится "утечкой данных", в результате чего модель будет показывать на тестовой/валидационной части данных результаты лучше, чем они есть на самом деле.

Ну и кроме того, как уже было упомянуто, даже скейлеры и импутеры обучаются. Скейлер может запомнить минимум и максимум значений каждой фичи, импутер может запомнить, например, медианы фич. Поэтому на тренировочных данных делаем fit_transform - одновременно и обучаемся и преобразуем тренировочные данные. А на тестовых/валидационных данных делаем уже только transform - преобразуем данные согласно ранее запомненным параметрам, полученным на тренировочной выборке.

**ЗАДАНИЕ: Мы будем использовать модель "Elastic Net". Создайте экземпляр модели ElasticNet в scikit-learn, используя параметры по умолчанию.**

In [26]:
from sklearn.linear_model import ElasticNet
help(ElasticNet)
base_elastic_model = ElasticNet()

Help on class ElasticNet in module sklearn.linear_model._coordinate_descent:

class ElasticNet(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, sklearn.linear_model._base.LinearModel)
 |  ElasticNet(alpha=1.0, *, l1_ratio=0.5, fit_intercept=True, precompute=False, max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
 |  
 |  Linear regression with combined L1 and L2 priors as regularizer.
 |  
 |  Minimizes the objective function::
 |  
 |          1 / (2 * n_samples) * ||y - Xw||^2_2
 |          + alpha * l1_ratio * ||w||_1
 |          + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2
 |  
 |  If you are interested in controlling the L1 and L2 penalty
 |  separately, keep in mind that this is equivalent to::
 |  
 |          a * ||w||_1 + 0.5 * b * ||w||_2^2
 |  
 |  where::
 |  
 |          alpha = a + b and l1_ratio = a / (a + b)
 |  
 |  The parameter l1_ratio corresponds to alpha in the glmnet R package while
 |  alph

**ЗАДАНИЕ: Модель Elastic Net работает с двумя основными параметрами - alpha и L1_ratio. Создайте словарь с набором различных значений этих параметров, для поиска по сетке. Вы можете выбрать различные значения параметров, но имейте ввиду, что Ваши результаты могут не совпасть с результатами в блокноте с решениями.**

In [27]:
param_grid = {'alpha':[i/10 for i in range(1175,1222,1)],
              'l1_ratio':[ .99, 1]}

**ЗАДАНИЕ: С помощью scikit-learn создайте объект GridSearchCV и запустите поиск по сетке для нахождения наилучших параметров модели, используя обучающие данные (предварительно смасштабированные). [Для некоторых комбинаций параметров Вы можете получить предупреждения (warnings).](https://stackoverflow.com/questions/20681864/lasso-on-sklearn-does-not-converge)**

In [28]:
# число verbose выбирайте сами
from sklearn.model_selection import GridSearchCV
grid_model = GridSearchCV(estimator=base_elastic_model,
                          param_grid=param_grid,
                          scoring='neg_mean_squared_error',
                          cv=5,
                          verbose=2)

**ЗАДАНИЕ: Отобразите наилучшую комбинацию параметров для Вашей модели.**

In [29]:
grid_model.fit(X_train,y_train)

[CV] END ............................alpha=117.8, l1_ratio=1; total time=   0.1s
[CV] END ............................alpha=117.8, l1_ratio=1; total time=   0.1s
[CV] END ............................alpha=117.8, l1_ratio=1; total time=   0.2s
[CV] END ............................alpha=117.8, l1_ratio=1; total time=   0.1s
[CV] END ............................alpha=117.8, l1_ratio=1; total time=   0.1s
[CV] END .........................alpha=117.9, l1_ratio=0.99; total time=   0.0s
[CV] END .........................alpha=117.9, l1_ratio=0.99; total time=   0.0s
[CV] END .........................alpha=117.9, l1_ratio=0.99; total time=   0.0s
[CV] END .........................alpha=117.9, l1_ratio=0.99; total time=   0.0s
[CV] END .........................alpha=117.9, l1_ratio=0.99; total time=   0.0s
[CV] END ............................alpha=117.9, l1_ratio=1; total time=   0.1s
[CV] END ............................alpha=117.9, l1_ratio=1; total time=   0.1s
[CV] END ...................

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [117.5, 117.6, 117.7, 117.8, 117.9, 118.0,
                                   118.1, 118.2, 118.3, 118.4, 118.5, 118.6,
                                   118.7, 118.8, 118.9, 119.0, 119.1, 119.2,
                                   119.3, 119.4, 119.5, 119.6, 119.7, 119.8,
                                   119.9, 120.0, 120.1, 120.2, 120.3, 120.4, ...],
                         'l1_ratio': [0.99, 1]},
             scoring='neg_mean_squared_error', verbose=2)

Fitting 5 folds for each of 94 candidates, totalling 470 fits
[CV] END .........................alpha=117.5, l1_ratio=0.99; total time=   0.0s
[CV] END .........................alpha=117.5, l1_ratio=0.99; total time=   0.0s
[CV] END .........................alpha=117.5, l1_ratio=0.99; total time=   0.0s
[CV] END .........................alpha=117.5, l1_ratio=0.99; total time=   0.0s
[CV] END .........................alpha=117.5, l1_ratio=0.99; total time=   0.0s
[CV] END ............................alpha=117.5, l1_ratio=1; total time=   0.1s
[CV] END ............................alpha=117.5, l1_ratio=1; total time=   0.1s
[CV] END ............................alpha=117.5, l1_ratio=1; total time=   0.1s
[CV] END ............................alpha=117.5, l1_ratio=1; total time=   0.1s
[CV] END ............................alpha=117.5, l1_ratio=1; total time=   0.1s
[CV] END .........................alpha=117.6, l1_ratio=0.99; total time=   0.0s
[CV] END .........................alpha=117.6, 

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [117.5, 117.6, 117.7, 117.8, 117.9, 118.0,
                                   118.1, 118.2, 118.3, 118.4, 118.5, 118.6,
                                   118.7, 118.8, 118.9, 119.0, 119.1, 119.2,
                                   119.3, 119.4, 119.5, 119.6, 119.7, 119.8,
                                   119.9, 120.0, 120.1, 120.2, 120.3, 120.4, ...],
                         'l1_ratio': [0.99, 1]},
             scoring='neg_mean_squared_error', verbose=2)

In [30]:
grid_model.best_estimator_

ElasticNet(alpha=118.0, l1_ratio=1)

ElasticNet(alpha=118.0, l1_ratio=1)

**ЗАДАНИЕ: Оцените работу модели на тестовом наборе данных (предварительно смасштабированном) в 10%, которые модель ещё не видела. В блокноте с решениями мы получили MAE = $\$$14149 и RMSE = $\$$20532**

In [31]:
y_pred = grid_model.predict(X_test)

In [32]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

In [33]:
sqrt(mean_squared_error(y_test,y_pred))

20557.40371552226

20557.40371552226

In [34]:
mean_absolute_error(y_test,y_pred)

14186.877607627428

14186.877607627428

## Отличная работа!

----